In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw13.ipynb")

# Homework 13: k-Means Clustering

<!-- BEGIN QUESTION -->

**What is your name?**

_Type your answer here, replacing this text._

<!-- END QUESTION -->



In [ ]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *
import sklearn
import sklearn.datasets
import sklearn.cluster

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

**Comment on Randomness**

Computer cannot really do random things.
When we ask it to give a random number or make a random choice, it uses a [Pseudorandom number generator](https://en.wikipedia.org/wiki/Pseudorandom_number_generator) to produce something that is random-like.
Such pseudorandom number generators start with a number called a `seed`.
If you reset the seed manually, then you will always get the same random-like number.
We use this fact in exercises so that your *random* objects are the same as the ones used when writing this assignment. The cells below illustrate how this works.


In [ ]:
# set the seed and get a random-like number
seed = 12345
rng = np.random.default_rng(seed)
rng.random()

In [ ]:
# doing it again gives the same number
seed = 12345
rng = np.random.default_rng(seed)
rng.random()

In [ ]:
# without re-setting the seed, a new random-like number is given
rng.random()

In [ ]:
# without re-setting the seed, a new random-like number is given
rng.random()

## 1. One Cluster

Let's make a single cluster and visualize it.

In [ ]:
X, true_labels = sklearn.datasets.make_blobs(n_samples=300, centers=1,
                       cluster_std=0.60, random_state=0)
one_tbl = Table().with_columns('x',X[:,0],'y',X[:,1])
one_tbl.scatter('x','y',s=50)
one_tbl

We can ask `sklearn` to cluster our data.
Using only 1 cluster, this is a bit silly, but works.

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=1)
kmeans.fit(X)
one_centers = kmeans.cluster_centers_
one_center = one_centers[0]
one_tbl.scatter('x','y',s=50)
plt.scatter(one_center[0], one_center[1], c='red', s=200, alpha=0.5)
one_center

**Question 1.1.**

Write a function to compute the mean of the coordinates. Then run it on `one_tbl`.

In [ ]:
def mean_of_cluster(cluster_tbl):
    x_mean = ...
    y_mean = ...
    return make_array(x_mean,y_mean)

one_tbl_mean = mean_of_cluster(one_tbl)
one_tbl_mean    

In [ ]:
grader.check("q1_1")

**Question 1.2**

Is this mean the same as the center computed by `sklearn`?
Set `center_is_mean` to `True` or `False`.

In [ ]:
center_is_mean = ...

In [ ]:
grader.check("q1_2")

**Question 1.3**

We can measure how spread out a cluster is by measuring the *Within Cluster Sum of Squares* (WCSS).
If the coordinates of the points are $\{(x_i,y_i)\}$ and the coordinates of the center is $(x_*,y_*)$, then this is given by the formula
$$ \sum_i \left((x_i-x_*)^2+(y_i-y_*)^2\right)\,.$$

Write a function that computes the WCSS for a single cluster.


In [ ]:
def within_cluster_sumof_squares(cluster_tbl,center):
    x_data = cluster_tbl.column('x')
    y_data = cluster_tbl.column('y')
    x_center = center[0]
    y_center = center[1]
    x_distance_squared = ...
    y_distance_squared = ...
    sum_distance_squared = ...
    return sum_distance_squared

one_WCSS = within_cluster_sumof_squares(one_tbl,one_center)
one_WCSS    

In [ ]:
grader.check("q1_3")

The center chosen by k-means clustering minimizes the WCSS.
In the following plot, we compute the WCSS for different proposed centers. 

In [ ]:
delta = 1
x_pts = np.linspace(one_center[0]-delta,one_center[0]+delta,100)
y_pts = np.linspace(one_center[1]-delta,one_center[1]+delta,100)
z_wcss = np.array([[within_cluster_sumof_squares(one_tbl,(x,y)) for x in x_pts] for y in y_pts])
plt.contour(x_pts,y_pts,z_wcss)
plt.colorbar()
plt.scatter(one_center[0], one_center[1], c='red', s=200, alpha=0.5)


## 2. Four Clusters
(This section is based on [In Depth: k-Means Clustering](https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html).)

Now let us try four clusters.

In [ ]:
four_X, true_labels = sklearn.datasets.make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
four_tbl = Table().with_columns('x',four_X[:,0],'y',four_X[:,1])
four_tbl.scatter('x','y',s=50)

We can ask `sklearn` to make four clusters.
In this case, the centers nicely match what we think the clusters should be.

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=4)
kmeans.fit(four_X)
four_centers = kmeans.cluster_centers_
four_tbl.scatter('x','y',s=50)
plt.scatter(four_centers[:,0], four_centers[:,1], c='red', s=200, alpha=0.5)
four_centers

We can also find out to which cluster each point has been allocated.
We will add this to our table.

In [ ]:
four_tbl_clustered = four_tbl.with_column("cluster",kmeans.predict(four_X))
four_tbl_clustered

We can then produce a nice plot colored by the cluster number.

In [ ]:
four_tbl_clustered.scatter("x","y",group="cluster")
plt.scatter(four_centers[:,0], four_centers[:,1], c='red', s=200, alpha=0.5)

**Question 2.1**

Use the function `within_cluster_sumof_squares` that you wrote earlier to compute the WCSS for each cluster and return them as an array.

In [ ]:
def all_wcss(cluster_tbl,centers):
    all_wcss = make_array()
    for i in np.arange(len(centers)):
        this_cluster_tbl = ...
        this_center = ...
        this_wcss = ...
        all_wcss = np.append(all_wcss,this_wcss)
    return all_wcss

four_all_wcss = all_wcss(four_tbl_clustered,four_centers)
four_all_wcss

In [ ]:
grader.check("q2_1")

It is possible for the k-means clustering algorithm to get stuck in a local minimum and not find reasonable clusters.

In [ ]:
kmeans2 = sklearn.cluster.KMeans(n_clusters=4,n_init=1,init=[[-1.37,7.75],[1.95,1.46],[-0.35,3.6],[2.01,0.44]])
kmeans2.fit(four_X)
four_centers2 = kmeans2.cluster_centers_
four_tbl_clustered2 = four_tbl.with_column("cluster",kmeans2.predict(four_X))
four_tbl_clustered2.scatter("x","y",group="cluster")
plt.scatter(four_centers2[:,0], four_centers2[:,1], c='red', s=200, alpha=0.5)
four_centers2

<!-- BEGIN QUESTION -->

**Question 2.2**

Compute `all_WCSS` for `four_tbl_clustered` and `four_tbl_clustered2`.
What do these numbers tell you about which clustering worked better?

_Type your answer here, replacing this text._

In [ ]:
all_wcss_four_tbl_clustered = ...
all_wcss_four_tbl_clustered2 = ...

print("clustered gives",all_wcss_four_tbl_clustered)
print("clustered2 gives",all_wcss_four_tbl_clustered2)


<!-- END QUESTION -->

## 3. How Many Clusters?

In the above sections, we knew how many clusters there should be, so we could specify how many to use.
In general, one does not know how many clusters to use, and the data is in a high dimension so one cannot plot it to decide by eye.

One way to decide is to look at the total WCSS. We can define a function to compute this, using the function `all_wcss` that you defined above.

In [ ]:
def total_WCSS(cluster_tbl,centers):
    return sum(all_wcss(cluster_tbl,centers))

<!-- BEGIN QUESTION -->

**Question 3.1**

Plot the total WCSS as a function of k, using the data that we know has 4 clusters.

In [ ]:
klist = [1,2,3,4,5,6,7]
WCSS_array = make_array()
for k in klist:
    this_kmeans = ...
    this_kmeans.fit(four_X)
    this_centers = this_kmeans.cluster_centers_
    this_tbl_clustered = ...
    this_total_WCSS = ...
    WCSS_array = np.append(WCSS_array,this_total_WCSS)
plt.plot(klist,WCSS_array)
plt.ylim([0,plt.ylim()[1]])

<!-- END QUESTION -->

You should see that the graph has an *elbow* at 4, where it switches from decreasing rapidly to decreasing slowly.

As we provide more centers, the total WCSS should keep decreasing. 
We can try to compensate for this effect by multiplying by the number of centers.

The following plot does this, and should show a kink at 4.

In [ ]:
plt.plot(klist,[WCSS_array[i]*klist[i] for i in range(len(klist))])
plt.ylim([0,plt.ylim()[1]])

## Submission

<!-- BEGIN QUESTION -->

**Acknowledgements:**
If you received help from any other person, then describe here what help you received and from whom. 
If not, then say 'None'.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

It is forbidden to: 
* Use the work or programs from students who took this class (in any of its versions at any university).
* Use websites that claim to have solutions for this class (in any of its versions at any university).
* Directly copy from a classmate.

**Did you do anything forbidden while working on this assignment?**

Assign `did_forbidden` to either `True` or `False`.

In [ ]:
do_forbidden = ...

In [ ]:
grader.check("q_forbidden")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

After saving and exporting, submit the .zip file in BlackBoard

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()